In [4]:
import pandas as pd
import math
import numpy as np
import re
from pandas.io.json import json_normalize
import json

In [235]:
df = pd.read_csv('taco_burrito/tacos_burritos.csv')

In [236]:
df.head()

,id,name,address,categories,city,country,cuisines,dateAdded,dateUpdated,keys,...,menus.dateSeen,menus.description,menus.name,name.1,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province,websites
0,AVwc__cikufWRAb51QjV,Gambino's Pizza,610 Commercial St,"Restaurant Delivery Service,Restaurants,Pizza,...",Atchison,US,Restaurant Delivery Service,2016-03-02T11:49:34Z,2017-02-01T18:04:44Z,us/ks/atchison/610commercialst/-138126000,...,2017-02-01T18:04:44Z,NaN,Taco,Gambino's Pizza,66002,NaN,NaN,NaN,KS,http://www.gambinospizza.com
1,AVwc__cikufWRAb51QjV,Gambino's Pizza,610 Commercial St,"Restaurant Delivery Service,Restaurants,Pizza,...",Atchison,US,Restaurant Delivery Service,2016-03-02T11:49:34Z,2017-02-01T18:04:44Z,us/ks/atchison/610commercialst/-138126000,...,2016-03-02T11:49:34Z,"Beef, onion, cheddar cheese, lettuce, refried...",Taco,Gambino's Pizza,66002,NaN,NaN,NaN,KS,http://www.gambinospizza.com
2,AVwc__dF_7pvs4fz1Ncy,Diablo Grande Golf & Country Club,9521 Morton Davis Dr,"Golf Course, American Restaurant, and Resort",Patterson,US,NaN,2016-10-14T01:58:25Z,2016-10-14T01:58:25Z,us/grayson/patterson/9521mortondavisdr/-522005016,...,2016-10-14T01:58:25Z,"Beef barbacoa or chicken, spanish rice, pico d...",Smotherd Burrito,Diablo Grande Golf & Country Club,95363,NaN,NaN,NaN,Grayson,http://www.diablogrande.com/
3,AVwc__kCByjofQCxkdm-,Mcdonald's,10550 W I 25 Frontage Rd,Fast Food Restaurant,Longmont,US,NaN,2016-09-09T17:30:34Z,2016-10-04T05:15:17Z,us/niwot/longmont/10550wi25frontagerd/-1161002137,...,2016-09-09T17:30:34Z,NaN,Sausage Burrito,Mcdonald's,80504,NaN,NaN,NaN,Niwot,http://mcdonalds.com
4,AVwc__WtByjofQCxkdkR,Baja Fresh Mexican Grill,8495 W 3rd St,"Mexican Restaurant Mid-City West,Mexican Resta...",Los Angeles,US,Mexican,2016-05-19T11:50:25Z,2016-09-11T00:06:37Z,us/bicentennial/losangeles/8495w3rdst/-251199537,...,2016-05-19T11:50:25Z,"Wahoo, avocado, cabbage, pico de gallo and av...",Wahoo Tacogrilled,Baja Fresh Mexican Grill,90048,USD,0.0,25.0,Bicentennial,http://bajafresh.com


In [129]:
df.shape

(77260, 27)

In [15]:
df.categories.unique()

array(['Restaurant Delivery Service,Restaurants,Pizza,Pizza Place',
       'Golf Course, American Restaurant, and Resort',
       'Fast Food Restaurant', ...,
       'Sandwich Place,Restaurant,Caterers,Restaurants,Cafeterias,Breakfast Brunch & Lunch Restaurants,,Sandwich Place, Coffee Shop, and Breakfast Spot,Coffee Shop,Breakfast Spot,Sandwich Place, Coffee Shop, and Breakfast Spot Downtown Springfield',
       'Restaurant and Bar City Center',
       'American Restaurant,Bars,American Restaurants,Restaurant,Brew Pubs,Brewery and American Restaurant,Restaurants,Bars & Pubs,Banquet Halls & Reception Facilities,Breweries,Brewery,Bar & Grills,Taverns'],
      dtype=object)

In [22]:
df.cuisines.unique()

array(['Restaurant Delivery Service', nan, 'Mexican', ...,
       'Mexican,Latin,Spanish,Vegetarian Friendly,Gluten Free Options',
       'American,Cafe,Vegetarian Friendly,Gluten Free Options',
       'American,Cafe,Gluten Free Options,Vegetarian Friendly'],
      dtype=object)

In [39]:
df[df['cuisines'] == 'American,Cafe,Vegetarian Friendly,Gluten Free Options']

,id,address,categories,city,country,cuisines,dateAdded,dateUpdated,keys,latitude,longitude,menuPageURL,menus.amountMax,menus.amountMin,menus.category,menus.currency,menus.dateSeen,menus.description,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province,websites
77218,AVzRGBdZLD2H7whiWc_u,2263 Chestnut St,Restaurant,San Francisco,US,"American,Cafe,Vegetarian Friendly,Gluten Free ...",2017-06-22T18:37:15Z,2017-06-22T18:39:28Z,us/california/sanfrancisco/2263chestnutst/-206...,37.79997,-122.44044,NaN,NaN,NaN,NaN,USD,"2017-06-22T16:29:14.281Z,2017-06-22T17:45:33.4...","eggs, chorizo, onions, peppers, avocado, peppe...",Breakfast Burrito,Squat and Gobble,94123-2607,NaN,NaN,NaN,California,NaN


In [41]:
df.loc[77218, 'cuisines']

'American,Cafe,Vegetarian Friendly,Gluten Free Options'

In [54]:
def unique_cuisines(data):
    unique_cuisines = set()
    if pd.isnull(data):
        data = 'NaN'
    cuisine = {*data.split(',')}
    unique_cuisines = unique_cuisines | cuisine

In [60]:
df['cuisines'].unique()

array(['Restaurant Delivery Service', nan, 'Mexican', ...,
       'Mexican,Latin,Spanish,Vegetarian Friendly,Gluten Free Options',
       'American,Cafe,Vegetarian Friendly,Gluten Free Options',
       'American,Cafe,Gluten Free Options,Vegetarian Friendly'],
      dtype=object)

## Unique Cuisines part

In [71]:
def strip(data):
    data = data.lstrip()
    data = data.rstrip()
    return data

In [72]:
unique_cuisines = set()
for each in df['cuisines'].unique():    
    if pd.isnull(each):
        each = 'NaN'
    lst = each.split(',')
    cuisine = set(map(lstrip, lst))
    unique_cuisines.update(cuisine)

In [73]:
unique_cuisines

{'Afghan',
 'Afternoon Tea',
 'American',
 'American ',
 'American (New)',
 'American (Traditional)',
 'American Cuisine',
 'American Restaurants',
 'Antipasta',
 'Argentinean',
 'Asian',
 'Asian Fusion',
 'Asian Restaurants',
 'Asian fusion',
 'Asian/Pacific',
 'Australian',
 'Bagels',
 'Bakeries',
 'Bakery',
 'Bakery ',
 'Bar',
 'Bar / Lounge / Bottle Service',
 'Bar Food',
 'Barbecue',
 'Barbecue ',
 'Barbecue Restaurants',
 'Bars',
 'Beer Garden',
 'Belgian',
 'Beverages',
 'Bistro',
 'Bowling',
 'Brazilian',
 'Brazilian Steakhouse',
 'Breakfast',
 'Breakfast ',
 'Breakfast and Brunch',
 'Breakfast/Brunch',
 'Brew Pubs',
 'Brewery',
 'Brick Oven Pizza',
 'British',
 'Buffet',
 'Buffet Restaurants',
 'Buffets',
 'Burgers',
 'Burgers ',
 'Burgers / Hot Dogs',
 'Burgers Description',
 'Burmese',
 'Cafe',
 'Cafeterias',
 'Cajun',
 'Californian',
 'Californian ',
 'Calzones',
 'Canadian',
 'Caribbean',
 'Caribbean Restaurants',
 'Casinos',
 'Caterers',
 'Central American',
 'Chain Resta

In [74]:
unique_cuisines = list(unique_cuisines)

In [81]:
def count_occurence(data, row):
    count_dict = dict()
    if data in row:
        if data in count_dict.keys():
            count_dict[data] += 1
        else:
            count_dict[data] = 1
    return count_dict

In [79]:
for each in df['cuisines']:
    if pd.isnull(each):
        each = 'NaN'

In [82]:
for each in unique_cuisines:
    df['cuisines'].apply(lambda x: count_occurence(each, x))

TypeError: argument of type 'float' is not iterable

## Ratio of burrito to taco

In [98]:
count_taco = 0
count_burrito = 0
for data in df['menus.name']:
    if pd.isnull(data):
        data = 'NaN'
    if re.search(r'[Bb][Uu][Rr][Rr][Ii][Tt][Oo]', data):
        count_burrito += 1
    elif re.search(r'[Tt][Aa][Cc][Oo]', data):
        count_taco += 1

In [99]:
count_burrito

35746

In [100]:
count_taco

41514

In [103]:
count_burrito/count_taco

0.8610589198824493

In [112]:
df[(df['name'] == 'Subway') & (df['menus.name'] == 'Breakfast Burrito')]

,id,address,categories,city,country,cuisines,dateAdded,dateUpdated,keys,latitude,longitude,menuPageURL,menus.amountMax,menus.amountMin,menus.category,menus.currency,menus.dateSeen,menus.description,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province,websites
469,AVwc_mPLByjofQCxkZWg,12344 Barker Cypress Rd,"Sandwich Shops,Restaurants,Fast Food Restauran...",Cypress,US,"Sandwich Shops,Restaurants",2016-04-14T04:52:06Z,2016-05-31T13:40:24Z,us/tx/cypress/12344barkercypressrd/-891525969,29.957493,-95.67179,NaN,8.99,8.99,NaN,USD,2016-12-19T18:42:30Z,"A generous portion of scrambled eggs, potatoes...",Breakfast Burrito,Subway,77429,USD,0.0,25.0,TX,"http://www.subway.com,http://subway.com"


In [115]:
nullamount = df[(pd.isnull(df['menus.amountMin']) == True) & (pd.isnull(df['menus.amountMax']) == True)]

In [118]:
priceimputation = df[(df['menus.name'].isin(nullamount['menus.name'])) & (df['name'].isin(nullamount['name'])) & (pd.isnull(df['menus.amountMin']) == False)]

In [169]:
priceimputation2 = priceimputation

In [170]:
priceimputation2 = priceimputation[['menus.name', 'name', 'menus.amountMin']]

In [171]:
priceimputation2.head()

,menus.name,name,menus.amountMin
26,Super Shrimp Burrito,Taqueria Las Comadres,10.5
28,Wet Burrito Plate,Taqueria Las Comadres,8.5
30,Super Wet Burrito,Taqueria Las Comadres,8.0
31,Burrito Plate,Taqueria Las Comadres,7.8
34,Super Burrito,Taqueria Las Comadres,7.3


In [183]:
priceimputation2.to_csv('priceimputation.csv', index = False)

In [172]:
priceimputation2.shape

(9925, 3)

In [121]:
df_priceimputed = df

In [166]:
df_priceimputed.loc[1, 'menus.name']

'Taco'

In [173]:
priceimputation2[priceimputation2['menus.name'] == 'Taco']

,menus.name,name,menus.amountMin
790,Taco,Agave Maria Restaurant & Cantina,4.95
1220,Taco,El Mexsal,1.95
1403,Taco,Cookie's Chuck Wagon Inc,7.50
1621,Taco,El Farolito Mexican Restaurant,3.00
2032,Taco,La Hacienda Real,3.25
2218,Taco,Zócalo Mexican Grill & Tequilería,8.00
2414,Taco,Don Jose Mexican Restaurant,4.75
3032,Taco,Little Onion Mexican Restaurant,6.75
3383,Taco,Chevys Fresh Mex,3.99
3416,Taco,Los Cabos,1.75


In [175]:
priceimputation2.loc[priceimputation2['menus.name'] == 'Taco', 'menus.name']

790      Taco
1220     Taco
1403     Taco
1621     Taco
2032     Taco
2218     Taco
2414     Taco
3032     Taco
3383     Taco
3416     Taco
4021     Taco
4091     Taco
4499     Taco
4881     Taco
4894     Taco
5183     Taco
6511     Taco
6841     Taco
6928     Taco
6985     Taco
7070     Taco
7199     Taco
7263     Taco
7649     Taco
7893     Taco
7920     Taco
8129     Taco
8486     Taco
8692     Taco
8757     Taco
         ... 
64678    Taco
64719    Taco
65163    Taco
66181    Taco
66275    Taco
67306    Taco
67368    Taco
67592    Taco
68442    Taco
68766    Taco
69146    Taco
69248    Taco
70061    Taco
70154    Taco
71155    Taco
71238    Taco
71525    Taco
71774    Taco
71873    Taco
72639    Taco
73085    Taco
73581    Taco
74002    Taco
74151    Taco
74661    Taco
74832    Taco
74916    Taco
75580    Taco
75851    Taco
75958    Taco
Name: menus.name, Length: 211, dtype: object

In [181]:
for index, value in enumerate(df_priceimputed['menus.amountMin']):
    if pd.isnull(value):
#         if (df_priceimputed.loc[index, 'menus.name'] == priceimputation2.loc[priceimputation2['menus.name'] == df_priceimputed.loc[index, 'menus.name'], 'menus.name']) & (df_priceimputed.loc[index, 'name'] == priceimputation2.loc[priceimputation2['name'] == df_priceimputed.loc[index, 'name'], 'name']):
        df_priceimputed.loc[index, 'menus.amountMin'] = priceimputation2.loc[(priceimputation2[priceimputation2['menus.name'] == df_priceimputed.loc[index, 'menus.name']]) & (priceimputation2[priceimputation2['name'] == df_priceimputed.loc[index, 'name']]), 'menus.amountMin']
    else:
        df_priceimputed.loc[index, 'menus.amountMin'] = value

ValueError: Cannot index with multidimensional key

In [ ]:
df_priceimputed['menus.amountMin'] = df_priceimputed['menus.amountMin'].apply(lambda x: x = )

In [ ]:
df_priceimputed['menus.amountMin'] = df[(df['menus.name'].isin(priceimputation['menus.name']) == True) & ]

In [184]:
df[(df['menus.name'] == 'Reg. Burrito') & (df['name'] == 'Taqueria Guadalajara')]

,id,address,categories,city,country,cuisines,dateAdded,dateUpdated,keys,latitude,longitude,menuPageURL,menus.amountMax,menus.amountMin,menus.category,menus.currency,menus.dateSeen,menus.description,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province,websites
10228,AVwce-M5kufWRAb5voWk,4141 Elverta Rd,"Mexican Restaurant,Take Out Restaurants,Latin ...",Antelope,US,"Take Out Restaurants,Mexican",2015-10-21T17:44:05Z,2016-05-06T21:00:52Z,us/ca/antelope/4141elvertard/1040909429,38.713936,-121.367099,http://www.citysearch.com/profile/menu/4110547...,4.4,4.4,NaN,USD,"2016-05-06T21:00:52Z,2016-04-04T05:40:14Z","Served with rice, beans, salsa and meat",Reg. Burrito,Taqueria Guadalajara,95843,NaN,NaN,NaN,CA,http://pandaexpress.com
15501,AVwciywWkufWRAb5wVe3,4798 Mission St,"Mexican Restaurant,Burrito Place,Taco Place,Re...",San Francisco,US,Mexican Restaurants,2016-03-25T03:12:13Z,2017-07-03T10:40:06Z,us/ca/sanfrancisco/4798missionst/1040909429,37.721200,-122.437547,NaN,4.4,4.4,NaN,USD,2016-03-25T03:12:13Z,"Served with rice, beans, salsa and meat",Reg. Burrito,Taqueria Guadalajara,94112,USD,0.0,25.0,CA,http://www.taqueriasguadalajara.com
22422,AVwcove3ByjofQCxggp8,105 S Interstate 35,Mexican Restaurant,New Braunfels,US,Latin American Restaurants,2016-06-14T02:39:10Z,2016-10-03T22:01:46Z,us/tx/newbraunfels/105sinterstate35/1040909429,29.690190,-98.110329,NaN,4.4,4.4,NaN,USD,"2016-06-14T02:39:10Z,2016-06-27T13:57:59Z","Served with rice, beans, salsa and meat",Reg. Burrito,Taqueria Guadalajara,78130,USD,0.0,25.0,TX,NaN
23559,AVwcpUFnIN2L1WUfnUBV,12226 Ranch Road 620 N,Mexican Restaurant,Austin,US,NaN,2016-03-02T12:01:41Z,2017-02-08T23:22:50Z,us/tx/austin/12226ranchroad620n/1040909429,30.462379,-97.815628,NaN,4.4,4.4,NaN,USD,2016-03-02T12:01:41Z,"Served with rice, beans, salsa and meat",Reg. Burrito,Taqueria Guadalajara,78750,USD,0.0,25.0,TX,NaN
36404,AVwd3MvfByjofQCxtKHG,3146 24th St,"Mexican Restaurant,Restaurant",San Francisco,US,NaN,2015-10-23T04:18:02Z,2017-07-03T10:29:58Z,us/ca/sanfrancisco/314624thst/1040909429,NaN,NaN,NaN,4.4,4.4,NaN,USD,"2016-03-28T05:18:13Z,2015-10-23T04:18:02Z","Served with rice, beans, salsa and meat",Reg. Burrito,Taqueria Guadalajara,94110,USD,0.0,25.0,CA,NaN
67293,AVwdYCD3kufWRAb55QDw,2547 Goliad Rd,"Mexican Restaurant,Taco Place,Mexican Restaura...",San Antonio,US,NaN,2016-03-29T04:56:55Z,2017-07-06T18:51:19Z,us/tx/sanantonio/2547goliadrd/1040909429,NaN,NaN,NaN,4.4,4.4,NaN,USD,2016-03-29T04:56:55Z,"Served with rice, beans, salsa and meat",Reg. Burrito,Taqueria Guadalajara,78223,USD,0.0,25.0,TX,NaN
70437,AVweDDjYkufWRAb5_i8d,4517 N Tryon St,"Mexican Restaurant,Mexican Restaurants,Latin A...",Charlotte,US,Mexican Restaurants,2016-04-01T06:26:53Z,2016-04-03T12:00:38Z,us/nc/charlotte/4517ntryonst/1040909429,35.258360,-80.790700,NaN,4.4,4.4,NaN,USD,"2016-04-01T06:26:53Z,2016-04-03T12:00:38Z","Served with rice, beans, salsa and meat",Reg. Burrito,Taqueria Guadalajara,28213,NaN,NaN,NaN,NC,NaN


In [20]:
data = [json.loads(line) for line in open('taco_burrito/business.json', 'r', encoding = 'utf-8')]

In [21]:
def flatten_json(nested_json, exclude=['']):
    """Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
            exclude: Keys to exclude from output.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

In [22]:
yelp_df = pd.DataFrame([flatten_json(x) for x in data])

In [23]:
yelp_df.shape

(192609, 60)

In [13]:
yelp_df.columns

Index(['address', 'attributes', 'attributes_AcceptsInsurance',
       'attributes_AgesAllowed', 'attributes_Alcohol', 'attributes_Ambience',
       'attributes_BYOB', 'attributes_BYOBCorkage', 'attributes_BestNights',
       'attributes_BikeParking', 'attributes_BusinessAcceptsBitcoin',
       'attributes_BusinessAcceptsCreditCards', 'attributes_BusinessParking',
       'attributes_ByAppointmentOnly', 'attributes_Caters',
       'attributes_CoatCheck', 'attributes_Corkage',
       'attributes_DietaryRestrictions', 'attributes_DogsAllowed',
       'attributes_DriveThru', 'attributes_GoodForDancing',
       'attributes_GoodForKids', 'attributes_GoodForMeal',
       'attributes_HairSpecializesIn', 'attributes_HappyHour',
       'attributes_HasTV', 'attributes_Music', 'attributes_NoiseLevel',
       'attributes_Open24Hours', 'attributes_OutdoorSeating',
       'attributes_RestaurantsAttire', 'attributes_RestaurantsCounterService',
       'attributes_RestaurantsDelivery', 'attributes_Restau

In [24]:
yelp_df = yelp_df[yelp_df['categories'].str.contains('[Rr]estaurant', na=False)]

In [25]:
yelp_df

,address,attributes,attributes_AcceptsInsurance,attributes_AgesAllowed,attributes_Alcohol,attributes_Ambience,attributes_BYOB,attributes_BYOBCorkage,attributes_BestNights,attributes_BikeParking,...,hours_Tuesday,hours_Wednesday,is_open,latitude,longitude,name,postal_code,review_count,stars,state
1,30 Eglinton Avenue W,NaN,NaN,NaN,u'full_bar',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,False,...,9:0-0:0,9:0-0:0,1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,"10110 Johnston Rd, Ste 15",NaN,NaN,NaN,u'beer_and_wine',"{'romantic': False, 'intimate': False, 'touris...",NaN,NaN,NaN,True,...,NaN,17:30-21:30,1,35.092564,-80.859132,Musashi Japanese Restaurant,28210,170,4.0,NC
11,2450 E Indian School Rd,NaN,NaN,NaN,u'none',"{'romantic': False, 'intimate': False, 'touris...",NaN,NaN,NaN,NaN,...,7:0-0:0,7:0-0:0,1,33.495194,-112.028588,Taco Bell,85016,18,3.0,AZ
13,5981 Andrews Rd,NaN,NaN,NaN,u'none',None,NaN,NaN,NaN,True,...,10:0-0:0,10:0-0:0,1,41.708520,-81.359556,Marco's Pizza,44060,16,4.0,OH
17,"1775 E Tropicana Ave, Ste 29",NaN,NaN,NaN,u'full_bar',"{'romantic': True, 'intimate': False, 'tourist...",NaN,'no',NaN,NaN,...,NaN,NaN,0,36.100016,-115.128529,Carluccio's Tivoli Gardens,89119,40,4.0,NV
23,"Center Core - Food Court, Fl 3, Pittsburgh Int...",NaN,NaN,NaN,NaN,"{'touristy': False, 'hipster': False, 'romanti...",NaN,NaN,NaN,NaN,...,NaN,NaN,1,40.496177,-80.246011,Marathon Diner,15231,35,4.0,PA
25,6055 E Lake Mead Blvd,NaN,NaN,NaN,u'beer_and_wine',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,...,10:0-21:0,10:0-21:0,1,36.195615,-115.040529,Maria's Mexican Restaurant & Bakery,89156,184,4.5,NV
29,1170 Queen Street W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,...,8:0-21:0,8:0-21:0,1,43.642889,-79.425429,Bolt Fresh Bar,M6J 1J5,57,3.0,ON
32,1051 Bloor Street W,NaN,NaN,NaN,u'full_bar',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,...,9:0-18:0,9:0-18:0,0,43.660494,-79.432099,The Steady Cafe & Bar,M6H 1M4,29,3.5,ON
35,6401 Morrison Blvd,NaN,NaN,NaN,u'full_bar',NaN,NaN,NaN,NaN,NaN,...,11:0-22:0,11:0-23:0,0,35.156338,-80.831878,Manzetti's Tavern,28211,16,3.5,NC


In [19]:
yelp_df.shape

(59387, 60)

In [27]:
yelp_mexican = yelp_df[yelp_df['categories'].str.contains('[Mm]exican', na=False)]

In [28]:
yelp_mexican.shape

(4628, 60)

In [30]:
yelp_mexican.head(100)

,address,attributes,attributes_AcceptsInsurance,attributes_AgesAllowed,attributes_Alcohol,attributes_Ambience,attributes_BYOB,attributes_BYOBCorkage,attributes_BestNights,attributes_BikeParking,...,hours_Tuesday,hours_Wednesday,is_open,latitude,longitude,name,postal_code,review_count,stars,state
11,2450 E Indian School Rd,NaN,NaN,NaN,u'none',"{'romantic': False, 'intimate': False, 'touris...",NaN,NaN,NaN,NaN,...,7:0-0:0,7:0-0:0,1,33.495194,-112.028588,Taco Bell,85016,18,3.0,AZ
25,6055 E Lake Mead Blvd,NaN,NaN,NaN,u'beer_and_wine',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,...,10:0-21:0,10:0-21:0,1,36.195615,-115.040529,Maria's Mexican Restaurant & Bakery,89156,184,4.5,NV
96,1614 Queen Street E,NaN,NaN,NaN,u'none',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,...,11:0-23:0,11:0-23:0,1,43.666181,-79.316468,Burrito Bandidos,M4L 1G4,43,3.5,ON
146,"3140 S Gilbert Rd, Ste 3",NaN,NaN,NaN,u'none',"{'romantic': False, 'intimate': False, 'touris...",NaN,NaN,NaN,True,...,NaN,NaN,0,33.261325,-111.790815,New Mexican Grill,85286,84,4.0,AZ
156,"1770 W Horizon Ridge, Ste 100",NaN,NaN,NaN,u'beer_and_wine',"{'touristy': False, 'hipster': False, 'romanti...",NaN,NaN,NaN,False,...,9:0-15:0,9:0-14:0,1,36.010745,-115.064803,Served,89012,664,4.5,NV
172,1921 Queen Street E,NaN,NaN,NaN,'none',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,False,...,NaN,NaN,0,43.669044,-79.304454,Gourmet Gringos,M4L 1H3,29,3.5,ON
181,"3415 West Glendale Ave, Bldg 25A",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11:0-22:0,11:0-22:0,1,33.537164,-112.132926,Sushi Mocorito,85051,9,3.0,AZ
276,1930 W Buckeye Rd,NaN,NaN,NaN,u'none',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,NaN,...,NaN,NaN,1,33.436984,-112.101267,Santanas Mexican Food,85009,6,3.0,AZ
285,971 O'Connor Drive,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,...,11:0-21:0,11:0-21:0,1,43.708355,-79.310127,Mexico Lindo,M4K 1T1,6,4.5,ON
343,10624 S Eastern Ave,NaN,NaN,NaN,u'none',"{'touristy': False, 'hipster': True, 'romantic...",NaN,NaN,NaN,False,...,11:0-19:0,11:0-21:30,1,35.998220,-115.102246,China AAA,89052,149,4.5,NV


In [32]:
yelp_mexican.to_csv('yelp_mexican.csv', index=False)

In [56]:
df.head()

,id,address,categories,city,country,cuisines,dateAdded,dateUpdated,keys,latitude,...,menus.dateSeen,menus.description,menus.name,name,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province,websites
0,AVwc__cikufWRAb51QjV,610 Commercial St,"Restaurant Delivery Service,Restaurants,Pizza,...",Atchison,US,Restaurant Delivery Service,2016-03-02T11:49:34Z,2017-02-01T18:04:44Z,us/ks/atchison/610commercialst/-138126000,39.561570,...,2017-02-01T18:04:44Z,NaN,Taco,Gambino's Pizza,66002,NaN,NaN,NaN,KS,http://www.gambinospizza.com
1,AVwc__cikufWRAb51QjV,610 Commercial St,"Restaurant Delivery Service,Restaurants,Pizza,...",Atchison,US,Restaurant Delivery Service,2016-03-02T11:49:34Z,2017-02-01T18:04:44Z,us/ks/atchison/610commercialst/-138126000,39.561570,...,2016-03-02T11:49:34Z,"Beef, onion, cheddar cheese, lettuce, refried...",Taco,Gambino's Pizza,66002,NaN,NaN,NaN,KS,http://www.gambinospizza.com
2,AVwc__dF_7pvs4fz1Ncy,9521 Morton Davis Dr,"Golf Course, American Restaurant, and Resort",Patterson,US,NaN,2016-10-14T01:58:25Z,2016-10-14T01:58:25Z,us/grayson/patterson/9521mortondavisdr/-522005016,37.401855,...,2016-10-14T01:58:25Z,"Beef barbacoa or chicken, spanish rice, pico d...",Smotherd Burrito,Diablo Grande Golf & Country Club,95363,NaN,NaN,NaN,Grayson,http://www.diablogrande.com/
3,AVwc__kCByjofQCxkdm-,10550 W I 25 Frontage Rd,Fast Food Restaurant,Longmont,US,NaN,2016-09-09T17:30:34Z,2016-10-04T05:15:17Z,us/niwot/longmont/10550wi25frontagerd/-1161002137,NaN,...,2016-09-09T17:30:34Z,NaN,Sausage Burrito,Mcdonald's,80504,NaN,NaN,NaN,Niwot,http://mcdonalds.com
4,AVwc__WtByjofQCxkdkR,8495 W 3rd St,"Mexican Restaurant Mid-City West,Mexican Resta...",Los Angeles,US,Mexican,2016-05-19T11:50:25Z,2016-09-11T00:06:37Z,us/bicentennial/losangeles/8495w3rdst/-251199537,34.080724,...,2016-05-19T11:50:25Z,"Wahoo, avocado, cabbage, pico de gallo and av...",Wahoo Tacogrilled,Baja Fresh Mexican Grill,90048,USD,0.0,25.0,Bicentennial,http://bajafresh.com


In [115]:
anim = df[['id', 'city', 'province', 'dateAdded', 'postalCode']]

In [116]:
anim.head()

,id,city,province,dateAdded,postalCode
0,AVwc__cikufWRAb51QjV,Atchison,KS,2016-03-02T11:49:34Z,66002
1,AVwc__cikufWRAb51QjV,Atchison,KS,2016-03-02T11:49:34Z,66002
2,AVwc__dF_7pvs4fz1Ncy,Patterson,Grayson,2016-10-14T01:58:25Z,95363
3,AVwc__kCByjofQCxkdm-,Longmont,Niwot,2016-09-09T17:30:34Z,80504
4,AVwc__WtByjofQCxkdkR,Los Angeles,Bicentennial,2016-05-19T11:50:25Z,90048


In [117]:
anim['year'] = anim['dateAdded'].apply(lambda x: x[:4])

C:\Users\Vedant\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [118]:
anim['month'] = anim['dateAdded'].apply(lambda x: x[5:7])

C:\Users\Vedant\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [119]:
anim.head()

,id,city,province,dateAdded,postalCode,year,month
0,AVwc__cikufWRAb51QjV,Atchison,KS,2016-03-02T11:49:34Z,66002,2016,03
1,AVwc__cikufWRAb51QjV,Atchison,KS,2016-03-02T11:49:34Z,66002,2016,03
2,AVwc__dF_7pvs4fz1Ncy,Patterson,Grayson,2016-10-14T01:58:25Z,95363,2016,10
3,AVwc__kCByjofQCxkdm-,Longmont,Niwot,2016-09-09T17:30:34Z,80504,2016,09
4,AVwc__WtByjofQCxkdkR,Los Angeles,Bicentennial,2016-05-19T11:50:25Z,90048,2016,05


In [75]:
zips = pd.read_csv('taco_burrito/uszips.csv')

In [76]:
zips.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,all_county_weights,imprecise,military,timezone
0,601,18.18004,-66.75218,Adjuntas,PR,Puerto Rico,True,NaN,18570,111.4,72001,Adjuntas,"{'72001':99.43,'72141':0.57}",False,False,America/Puerto_Rico
1,602,18.36073,-67.17517,Aguada,PR,Puerto Rico,True,NaN,41520,523.5,72003,Aguada,{'72003':100},False,False,America/Puerto_Rico
2,603,18.45439,-67.12202,Aguadilla,PR,Puerto Rico,True,NaN,54689,667.9,72005,Aguadilla,{'72005':100},False,False,America/Puerto_Rico
3,606,18.16724,-66.93828,Maricao,PR,Puerto Rico,True,NaN,6615,60.4,72093,Maricao,"{'72093':94.88,'72121':1.35,'72153':3.78}",False,False,America/Puerto_Rico
4,610,18.29032,-67.12243,Anasco,PR,Puerto Rico,True,NaN,29016,312.0,72011,Añasco,"{'72003':0.55,'72011':99.45}",False,False,America/Puerto_Rico


In [84]:
zips = zips[['zip', 'state_id', 'city', 'population']]

In [120]:
anim['postalCode'] = anim['postalCode'].fillna(0)

C:\Users\Vedant\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [121]:
anim = anim.astype({'postalCode': 'str'})

In [122]:
anim['postalCode'] = anim['postalCode'].apply(lambda x: x[:5])

In [123]:
anim = anim.astype({'postalCode': 'int64'})

In [124]:
anim2 = pd.merge(anim, zips, how='left', left_on='postalCode', right_on='zip', )

In [125]:
anim2.head()

,id,city_x,province,dateAdded,postalCode,year,month,zip,state_id,city_y,population
0,AVwc__cikufWRAb51QjV,Atchison,KS,2016-03-02T11:49:34Z,66002,2016,03,66002.0,KS,Atchison,13755.0
1,AVwc__cikufWRAb51QjV,Atchison,KS,2016-03-02T11:49:34Z,66002,2016,03,66002.0,KS,Atchison,13755.0
2,AVwc__dF_7pvs4fz1Ncy,Patterson,Grayson,2016-10-14T01:58:25Z,95363,2016,10,95363.0,CA,Patterson,25237.0
3,AVwc__kCByjofQCxkdm-,Longmont,Niwot,2016-09-09T17:30:34Z,80504,2016,09,80504.0,CO,Longmont,46876.0
4,AVwc__WtByjofQCxkdkR,Los Angeles,Bicentennial,2016-05-19T11:50:25Z,90048,2016,05,90048.0,CA,Los Angeles,21397.0


In [126]:
anim2.to_csv('anim.csv', index=False)

In [221]:
zips.head()

,zip,state_id,city,population
0,601,PR,Adjuntas,18570
1,602,PR,Aguada,41520
2,603,PR,Aguadilla,54689
3,606,PR,Maricao,6615
4,610,PR,Anasco,29016


In [250]:
zips = zips.rename(columns = {'city' : 'city_updated'})

In [251]:
zips.head()

,zip,state_id,city_updated,population
0,601,PR,Adjuntas,18570
1,602,PR,Aguada,41520
2,603,PR,Aguadilla,54689
3,606,PR,Maricao,6615
4,610,PR,Anasco,29016


In [222]:
zips2 = zips[['zip', 'state_id']]

In [223]:
zips2.head()

,zip,state_id
0,601,PR
1,602,PR
2,603,PR
3,606,PR
4,610,PR


In [237]:
df2 = df

In [238]:
df2.head()

,id,name,address,categories,city,country,cuisines,dateAdded,dateUpdated,keys,...,menus.dateSeen,menus.description,menus.name,name.1,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province,websites
0,AVwc__cikufWRAb51QjV,Gambino's Pizza,610 Commercial St,"Restaurant Delivery Service,Restaurants,Pizza,...",Atchison,US,Restaurant Delivery Service,2016-03-02T11:49:34Z,2017-02-01T18:04:44Z,us/ks/atchison/610commercialst/-138126000,...,2017-02-01T18:04:44Z,NaN,Taco,Gambino's Pizza,66002,NaN,NaN,NaN,KS,http://www.gambinospizza.com
1,AVwc__cikufWRAb51QjV,Gambino's Pizza,610 Commercial St,"Restaurant Delivery Service,Restaurants,Pizza,...",Atchison,US,Restaurant Delivery Service,2016-03-02T11:49:34Z,2017-02-01T18:04:44Z,us/ks/atchison/610commercialst/-138126000,...,2016-03-02T11:49:34Z,"Beef, onion, cheddar cheese, lettuce, refried...",Taco,Gambino's Pizza,66002,NaN,NaN,NaN,KS,http://www.gambinospizza.com
2,AVwc__dF_7pvs4fz1Ncy,Diablo Grande Golf & Country Club,9521 Morton Davis Dr,"Golf Course, American Restaurant, and Resort",Patterson,US,NaN,2016-10-14T01:58:25Z,2016-10-14T01:58:25Z,us/grayson/patterson/9521mortondavisdr/-522005016,...,2016-10-14T01:58:25Z,"Beef barbacoa or chicken, spanish rice, pico d...",Smotherd Burrito,Diablo Grande Golf & Country Club,95363,NaN,NaN,NaN,Grayson,http://www.diablogrande.com/
3,AVwc__kCByjofQCxkdm-,Mcdonald's,10550 W I 25 Frontage Rd,Fast Food Restaurant,Longmont,US,NaN,2016-09-09T17:30:34Z,2016-10-04T05:15:17Z,us/niwot/longmont/10550wi25frontagerd/-1161002137,...,2016-09-09T17:30:34Z,NaN,Sausage Burrito,Mcdonald's,80504,NaN,NaN,NaN,Niwot,http://mcdonalds.com
4,AVwc__WtByjofQCxkdkR,Baja Fresh Mexican Grill,8495 W 3rd St,"Mexican Restaurant Mid-City West,Mexican Resta...",Los Angeles,US,Mexican,2016-05-19T11:50:25Z,2016-09-11T00:06:37Z,us/bicentennial/losangeles/8495w3rdst/-251199537,...,2016-05-19T11:50:25Z,"Wahoo, avocado, cabbage, pico de gallo and av...",Wahoo Tacogrilled,Baja Fresh Mexican Grill,90048,USD,0.0,25.0,Bicentennial,http://bajafresh.com


In [239]:
df2['postalCode'] = df2['postalCode'].fillna('0')

In [240]:
df2['postalCode'] = df2['postalCode'].apply(lambda x: x[:5])

In [241]:
df2

,id,name,address,categories,city,country,cuisines,dateAdded,dateUpdated,keys,...,menus.dateSeen,menus.description,menus.name,name.1,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province,websites
0,AVwc__cikufWRAb51QjV,Gambino's Pizza,610 Commercial St,"Restaurant Delivery Service,Restaurants,Pizza,...",Atchison,US,Restaurant Delivery Service,2016-03-02T11:49:34Z,2017-02-01T18:04:44Z,us/ks/atchison/610commercialst/-138126000,...,2017-02-01T18:04:44Z,NaN,Taco,Gambino's Pizza,66002,NaN,NaN,NaN,KS,http://www.gambinospizza.com
1,AVwc__cikufWRAb51QjV,Gambino's Pizza,610 Commercial St,"Restaurant Delivery Service,Restaurants,Pizza,...",Atchison,US,Restaurant Delivery Service,2016-03-02T11:49:34Z,2017-02-01T18:04:44Z,us/ks/atchison/610commercialst/-138126000,...,2016-03-02T11:49:34Z,"Beef, onion, cheddar cheese, lettuce, refried...",Taco,Gambino's Pizza,66002,NaN,NaN,NaN,KS,http://www.gambinospizza.com
2,AVwc__dF_7pvs4fz1Ncy,Diablo Grande Golf & Country Club,9521 Morton Davis Dr,"Golf Course, American Restaurant, and Resort",Patterson,US,NaN,2016-10-14T01:58:25Z,2016-10-14T01:58:25Z,us/grayson/patterson/9521mortondavisdr/-522005016,...,2016-10-14T01:58:25Z,"Beef barbacoa or chicken, spanish rice, pico d...",Smotherd Burrito,Diablo Grande Golf & Country Club,95363,NaN,NaN,NaN,Grayson,http://www.diablogrande.com/
3,AVwc__kCByjofQCxkdm-,Mcdonald's,10550 W I 25 Frontage Rd,Fast Food Restaurant,Longmont,US,NaN,2016-09-09T17:30:34Z,2016-10-04T05:15:17Z,us/niwot/longmont/10550wi25frontagerd/-1161002137,...,2016-09-09T17:30:34Z,NaN,Sausage Burrito,Mcdonald's,80504,NaN,NaN,NaN,Niwot,http://mcdonalds.com
4,AVwc__WtByjofQCxkdkR,Baja Fresh Mexican Grill,8495 W 3rd St,"Mexican Restaurant Mid-City West,Mexican Resta...",Los Angeles,US,Mexican,2016-05-19T11:50:25Z,2016-09-11T00:06:37Z,us/bicentennial/losangeles/8495w3rdst/-251199537,...,2016-05-19T11:50:25Z,"Wahoo, avocado, cabbage, pico de gallo and av...",Wahoo Tacogrilled,Baja Fresh Mexican Grill,90048,USD,0.0,25.0,Bicentennial,http://bajafresh.com
5,AVwc__WtByjofQCxkdkR,Baja Fresh Mexican Grill,8495 W 3rd St,"Mexican Restaurant Mid-City West,Mexican Resta...",Los Angeles,US,Mexican,2016-05-19T11:50:25Z,2016-09-11T00:06:37Z,us/bicentennial/losangeles/8495w3rdst/-251199537,...,2016-05-19T11:50:25Z,"Hand breaded fried wahoo, topped with cabbag...",Wahoo Tacocrispy,Baja Fresh Mexican Grill,90048,USD,0.0,25.0,Bicentennial,http://bajafresh.com
6,AVwc__WtByjofQCxkdkR,Baja Fresh Mexican Grill,8495 W 3rd St,"Mexican Restaurant Mid-City West,Mexican Resta...",Los Angeles,US,Mexican,2016-05-19T11:50:25Z,2016-09-11T00:06:37Z,us/bicentennial/losangeles/8495w3rdst/-251199537,...,2016-05-19T11:50:25Z,Any combination of two original baja or americ...,Taco Combo,Baja Fresh Mexican Grill,90048,USD,0.0,25.0,Bicentennial,http://bajafresh.com
7,AVwc__WtByjofQCxkdkR,Baja Fresh Mexican Grill,8495 W 3rd St,"Mexican Restaurant Mid-City West,Mexican Resta...",Los Angeles,US,Mexican,2016-05-19T11:50:25Z,2016-09-11T00:06:37Z,us/bicentennial/losangeles/8495w3rdst/-251199537,...,2016-05-19T11:50:25Z,"Grilled corn tortillas with salsa, onions and ...",Original Baja Taco,Baja Fresh Mexican Grill,90048,USD,0.0,25.0,Bicentennial,http://bajafresh.com
8,AVwc__WtByjofQCxkdkR,Baja Fresh Mexican Grill,8495 W 3rd St,"Mexican Restaurant Mid-City West,Mexican Resta...",Los Angeles,US,Mexican,2016-05-19T11:50:25Z,2016-09-11T00:06:37Z,us/bicentennial/losangeles/8495w3rdst/-251199537,...,2016-05-19T11:50:25Z,NaN,Grilled Veggie Burrito,Baja Fresh Mexican Grill,90048,USD,0.0,25.0,Bicentennial,http://bajafresh.com
9,AVwc__WtByjofQCxkdkR,Baja Fresh Mexican Grill,8495 W 3rd St,"Mexican Restaurant Mid-City West,Mexican Resta...",Los Angeles,US,Mexican,2016-05-19T11:50:25Z,2016-09-11T00:06:37Z,us/bicentennial/losangeles/8495w3rdst/-251199537,...,2016-05-19T11:50:25Z,Cabo style or caesar style.,Grilled Chicken Salad Burrito,Baja Fresh Mexican Grill,90048,USD,0.0,25.0,Bicentennial,http://bajafresh.com


In [246]:
df2 = df2.astype({'postalCode' : 'int64'})

In [247]:
df3 = pd.merge(df2, zips2, how='left', left_on='postalCode', right_on='zip')

In [248]:
df3.head()

,id,name,address,categories,city,country,cuisines,dateAdded,dateUpdated,keys,...,menus.name,name.1,postalCode,priceRangeCurrency,priceRangeMin,priceRangeMax,province,websites,zip,state_id
0,AVwc__cikufWRAb51QjV,Gambino's Pizza,610 Commercial St,"Restaurant Delivery Service,Restaurants,Pizza,...",Atchison,US,Restaurant Delivery Service,2016-03-02T11:49:34Z,2017-02-01T18:04:44Z,us/ks/atchison/610commercialst/-138126000,...,Taco,Gambino's Pizza,66002,NaN,NaN,NaN,KS,http://www.gambinospizza.com,66002.0,KS
1,AVwc__cikufWRAb51QjV,Gambino's Pizza,610 Commercial St,"Restaurant Delivery Service,Restaurants,Pizza,...",Atchison,US,Restaurant Delivery Service,2016-03-02T11:49:34Z,2017-02-01T18:04:44Z,us/ks/atchison/610commercialst/-138126000,...,Taco,Gambino's Pizza,66002,NaN,NaN,NaN,KS,http://www.gambinospizza.com,66002.0,KS
2,AVwc__dF_7pvs4fz1Ncy,Diablo Grande Golf & Country Club,9521 Morton Davis Dr,"Golf Course, American Restaurant, and Resort",Patterson,US,NaN,2016-10-14T01:58:25Z,2016-10-14T01:58:25Z,us/grayson/patterson/9521mortondavisdr/-522005016,...,Smotherd Burrito,Diablo Grande Golf & Country Club,95363,NaN,NaN,NaN,Grayson,http://www.diablogrande.com/,95363.0,CA
3,AVwc__kCByjofQCxkdm-,Mcdonald's,10550 W I 25 Frontage Rd,Fast Food Restaurant,Longmont,US,NaN,2016-09-09T17:30:34Z,2016-10-04T05:15:17Z,us/niwot/longmont/10550wi25frontagerd/-1161002137,...,Sausage Burrito,Mcdonald's,80504,NaN,NaN,NaN,Niwot,http://mcdonalds.com,80504.0,CO
4,AVwc__WtByjofQCxkdkR,Baja Fresh Mexican Grill,8495 W 3rd St,"Mexican Restaurant Mid-City West,Mexican Resta...",Los Angeles,US,Mexican,2016-05-19T11:50:25Z,2016-09-11T00:06:37Z,us/bicentennial/losangeles/8495w3rdst/-251199537,...,Wahoo Tacogrilled,Baja Fresh Mexican Grill,90048,USD,0.0,25.0,Bicentennial,http://bajafresh.com,90048.0,CA


In [249]:
df3.to_csv('tacos_burritos_updated.csv', index=False)

In [252]:
df4 = pd.merge(df2, zips, how='left', left_on='postalCode', right_on='zip')

In [253]:
df4.to_csv('tacos_burritos_updated_2_0.csv', index=False)

In [254]:
df3.priceRangeMin.unique()

array([ nan,   0.,  25.,  50.,  31.,   1., 127., 169.,  10.,   7., 378.,
         5.,  12.,  11.,   3.,   8.,  15., 165., 115., 356.,  79.,   2.,
         6., 219.,  70.,  18.])

In [255]:
df3.priceRangeMax.unique()

array([ nan,  25.,  30.,  40.,  55.,  50.,  10.,   9., 222., 383.,  20.,
        12., 755.,  15.,   7.,  18.,   8., 166., 205., 483., 189.,  14.,
         5., 299., 280.])

In [259]:
df4['priceRangeMax'] = df4['priceRangeMax'].fillna(0)

In [260]:
df4['priceRangeMin'] = df4['priceRangeMin'].fillna(0)

In [261]:
for each in range(df4.shape[0]):
    if (df4.loc[each, 'priceRangeMin'] >= 0) and (df4.loc[each, 'priceRangeMax'] <= 20) and (df4.loc[each, 'priceRangeMin'] < df4.loc[each, 'priceRangeMax']):
        df4.loc[each, 'dollar_sign'] = 1
    elif (df4.loc[each, 'priceRangeMin'] > 20) and (df4.loc[each, 'priceRangeMax'] <= 40) and (df4.loc[each, 'priceRangeMin'] < df4.loc[each, 'priceRangeMax']):
        df4.loc[each, 'dollar_sign'] = 2
    elif (df4.loc[each, 'priceRangeMin'] > 40) and (df4.loc[each, 'priceRangeMax'] <= 100) and (df4.loc[each, 'priceRangeMin'] < df4.loc[each, 'priceRangeMax']):
        df4.loc[each, 'dollar_sign'] = 3
    elif (df4.loc[each, 'priceRangeMin'] > 100) and (df4.loc[each, 'priceRangeMax'] <= 150) and (df4.loc[each, 'priceRangeMin'] < df4.loc[each, 'priceRangeMax']):
        df4.loc[each, 'dollar_sign'] = 4
    elif (df4.loc[each, 'priceRangeMin'] > 150) and (df4.loc[each, 'priceRangeMax'] <= 5000) and (df4.loc[each, 'priceRangeMin'] < df4.loc[each, 'priceRangeMax']):
        df4.loc[each, 'dollar_sign'] = 5

In [262]:
df4.head()

,id,name,address,categories,city,country,cuisines,dateAdded,dateUpdated,keys,...,priceRangeCurrency,priceRangeMin,priceRangeMax,province,websites,zip,state_id,city_updated,population,dollar_sign
0,AVwc__cikufWRAb51QjV,Gambino's Pizza,610 Commercial St,"Restaurant Delivery Service,Restaurants,Pizza,...",Atchison,US,Restaurant Delivery Service,2016-03-02T11:49:34Z,2017-02-01T18:04:44Z,us/ks/atchison/610commercialst/-138126000,...,NaN,0.0,0.0,KS,http://www.gambinospizza.com,66002.0,KS,Atchison,13755.0,NaN
1,AVwc__cikufWRAb51QjV,Gambino's Pizza,610 Commercial St,"Restaurant Delivery Service,Restaurants,Pizza,...",Atchison,US,Restaurant Delivery Service,2016-03-02T11:49:34Z,2017-02-01T18:04:44Z,us/ks/atchison/610commercialst/-138126000,...,NaN,0.0,0.0,KS,http://www.gambinospizza.com,66002.0,KS,Atchison,13755.0,NaN
2,AVwc__dF_7pvs4fz1Ncy,Diablo Grande Golf & Country Club,9521 Morton Davis Dr,"Golf Course, American Restaurant, and Resort",Patterson,US,NaN,2016-10-14T01:58:25Z,2016-10-14T01:58:25Z,us/grayson/patterson/9521mortondavisdr/-522005016,...,NaN,0.0,0.0,Grayson,http://www.diablogrande.com/,95363.0,CA,Patterson,25237.0,NaN
3,AVwc__kCByjofQCxkdm-,Mcdonald's,10550 W I 25 Frontage Rd,Fast Food Restaurant,Longmont,US,NaN,2016-09-09T17:30:34Z,2016-10-04T05:15:17Z,us/niwot/longmont/10550wi25frontagerd/-1161002137,...,NaN,0.0,0.0,Niwot,http://mcdonalds.com,80504.0,CO,Longmont,46876.0,NaN
4,AVwc__WtByjofQCxkdkR,Baja Fresh Mexican Grill,8495 W 3rd St,"Mexican Restaurant Mid-City West,Mexican Resta...",Los Angeles,US,Mexican,2016-05-19T11:50:25Z,2016-09-11T00:06:37Z,us/bicentennial/losangeles/8495w3rdst/-251199537,...,USD,0.0,25.0,Bicentennial,http://bajafresh.com,90048.0,CA,Los Angeles,21397.0,NaN


In [263]:
df4.dollar_sign.unique()

array([nan,  2.,  3.,  1.,  5.])

In [264]:
df5 = df4

In [265]:
df5.to_csv('tacos_burritos_3_0.csv', index = False)